# Final Capstone Assignment

## Introduction to EmmiGREATion Pty Ltd and Business Problem

Emmigreation is a company that was founded to help people move seemlessly from one city to another. The company has noticed that when previous clients have moved to new neighbourhoods that happen to show some similarity to their previous home, they were far happier than those who moved to neighbourhoods that showed no similarities. This has led to Emmigreation hiring an IBM Data Scientist to determine if there was a way that the company could propose specific neighbourhoods to their clients to help achieve this outcome using a more scientific approach. The following report will determine how this can be done as well as going through an existing client's options for moving into a new city.

## Data  

The data that will be analysed will be the neighbourhoods of Toronto together with the neighbourhoods of New York. The data sets will be acquired the same way as before in the IBM Capstone course including the coordinates of each of the neighbourhoods. Using the Foursqaure API, we will be able link the closest/most-common amenities to these neighbourhoods to build up the data set required for the analysis. Once we have each neighbourhood from Toronto and New York City in one data set, with associated coordinates plus top 10 most common amenities within a certain radius, then we will be ready to run our analysis.


## Methodology

The methodology will use web scraping to pull data for Toronto and New York City neighbourhoods. The latitudes and longitudes of these neighbourhoods will then be linked and turned into a dataframe for analysis. The Foursquare API will then be utilised to link nearby amenities to the different neighbourhoods, which will then begin to build up profiles/characteristics for each neighbourhood. Using this dataset, we will then run a k-means clustering machine learning algorithm to determine which of the neighburhoods are similar. From this dataset, we can determine which Toronto neighbourhoods are similar to those of New York City. Emmigreation will be able to use this dataset to guide its clients on where they should consider moving to when moving in between these cities. This will achieve the outcomes set out in the introduction above.

Lastly, we will look at one specific example of a client wanting to move from one neighbourhood to another across the aforementioned cities.

### Importing necessary Python packages

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import lxml.html as lh #webscraping package

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  10.52 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  30.54 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  32.10 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  41.51 MB/s
Libraries imported.


### Scraping web for Toronto data to build dataset

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [6]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()

    col.append((name,[]))

In [7]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [8]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [9]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [10]:
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [11]:
df.rename(columns={'Neighbourhood\n' : 'Neighborhood'},inplace = True)

In [12]:
list(df)

['Borough', 'Neighborhood', 'Postcode']

In [13]:
df['Neighborhood']  = df['Neighborhood'].str.replace('\n','')

In [14]:
df.head()

,Borough,Neighborhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


### drop any rows with 'Not assigned' Borough

In [15]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)

### Replace not assigned Neighburhoods with Borough Name

In [16]:
df['Neighborhood'] = df['Neighborhood'].replace('Not assigned',df['Borough'])
df.head()

,Borough,Neighborhood,Postcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A


### Gets unique postal codes and creates list of neighbourhoods where there are mutliple

In [17]:
toronto_data = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
toronto_data.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Adds lats and longs

In [18]:
toronto_latlongs = pd.read_csv('http://cocl.us/Geospatial_data')
print('Data downloaded!') 

Data downloaded!


In [19]:
toronto = pd.merge(toronto_data, toronto_latlongs, how='left', left_on=('Postcode'), right_on=('Postal Code'))
toronto.head()

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [20]:
toronto.drop(columns='Postal Code', inplace=True)


In [21]:
toronto['City'] = 'Toronto'

In [22]:
toronto.drop(columns='Postcode', inplace=True)

In [23]:
toronto

,Borough,Neighborhood,Latitude,Longitude,City
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Toronto
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Toronto
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Toronto
3,Scarborough,Woburn,43.770992,-79.216917,Toronto
4,Scarborough,Cedarbrae,43.773136,-79.239476,Toronto
5,Scarborough,Scarborough Village,43.744734,-79.239476,Toronto
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Toronto
7,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Toronto
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,Toronto
9,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,Toronto


## NYC Dataset

Pulling data from url 

In [24]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [25]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [26]:
neighborhoods_data = newyork_data['features']

In [27]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [28]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [29]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [30]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [31]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [32]:
newyork = neighborhoods

In [33]:
newyork['City'] = 'New York'

In [34]:
newyork.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Bronx,Wakefield,40.894705,-73.847201,New York
1,Bronx,Co-op City,40.874294,-73.829939,New York
2,Bronx,Eastchester,40.887556,-73.827806,New York
3,Bronx,Fieldston,40.895437,-73.905643,New York
4,Bronx,Riverdale,40.890834,-73.912585,New York


### We will now concatenate the two tables to form one final dataset on which we can begin to add the foursquare data and then run the k-means clustering

In [35]:
nyc_toronto = pd.concat([toronto, newyork])

In [36]:
nyc_toronto

,Borough,Neighborhood,Latitude,Longitude,City
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Toronto
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Toronto
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Toronto
3,Scarborough,Woburn,43.770992,-79.216917,Toronto
4,Scarborough,Cedarbrae,43.773136,-79.239476,Toronto
5,Scarborough,Scarborough Village,43.744734,-79.239476,Toronto
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Toronto
7,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Toronto
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,Toronto
9,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,Toronto


## Getting Foursquare Data

In [37]:
CLIENT_ID = 'ESF4ZTQWDK4M0KWFYBLHFZEOZSJBRVX0JOC42VPIH3GDDV0H' # your Foursquare ID
CLIENT_SECRET = 'NFLTJE3UEMHFIRCZPIARJT2TDI2OBT5OLUZIXVOVPHZWEX3O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ESF4ZTQWDK4M0KWFYBLHFZEOZSJBRVX0JOC42VPIH3GDDV0H
CLIENT_SECRET:NFLTJE3UEMHFIRCZPIARJT2TDI2OBT5OLUZIXVOVPHZWEX3O


In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
# type your answer here

all_venues = getNearbyVenues(names=nyc_toronto['Neighborhood'],
                                   latitudes=nyc_toronto['Latitude'],
                                   longitudes=nyc_toronto['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [42]:
all_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [43]:
# one hot encoding
allvenues_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
allvenues_onehot['Neighborhood'] = all_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [allvenues_onehot.columns[-1]] + list(allvenues_onehot.columns[:-1])
allvenues_onehot = allvenues_onehot[fixed_columns]

allvenues_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous

In [45]:
allvenues_grouped = allvenues_onehot.groupby('Neighborhood').mean().reset_index()
allvenues_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,

In [46]:
allvenues_grouped.shape

(399, 461)

## Determining Top 10 most common venues

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = allvenues_grouped['Neighborhood']

for ind in np.arange(allvenues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(allvenues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Thai Restaurant,Steakhouse,Café,Bar,Gym,Burger Joint,Bakery,American Restaurant,Restaurant
1,Agincourt,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Women's Store,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Sandwich Place,Coffee Shop,Pharmacy,Fried Chicken Joint,Fast Food Restaurant,Beer Store,Pizza Place,Event Space,Exhibit
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Pub,Coffee Shop,Pool,Sandwich Place,Pharmacy,Gym,Egyptian Restaurant,Electronics Store


In [50]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Thai Restaurant,Steakhouse,Café,Bar,Gym,Burger Joint,Bakery,American Restaurant,Restaurant
1,Agincourt,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Women's Store,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Sandwich Place,Coffee Shop,Pharmacy,Fried Chicken Joint,Fast Food Restaurant,Beer Store,Pizza Place,Event Space,Exhibit
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Pub,Coffee Shop,Pool,Sandwich Place,Pharmacy,Gym,Egyptian Restaurant,Electronics Store
5,Allerton,Pizza Place,Supermarket,Spa,Intersection,Chinese Restaurant,Fried Chicken Joint,Breakfast Spot,Spanish Restaurant,Fast Food Restaurant,Pharmacy
6,Annadale,Pizza Place,Sports Bar,Pub,Train Station,Deli / Bodega,American Restaurant,Dance Studio,Diner,Restaurant,Food
7,Arden Heights,Pharmacy,Pizza Place,Coffee Shop,Duty-free Shop,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
8,Arlington,Bus Stop,Grocery Store,Coffee Shop,Tree,American Restaurant,Deli / Bodega,Playground,Boat or Ferry,Event Space,Exhibit
9,Arrochar,Liquor Store,Pizza Place,Bus Stop,Italian Restaurant,Deli / Bodega,Hotel,Athletics & Sports,Outdoors & Recreation,Mediterranean Restaurant,Middle Eastern Restaurant


## Clustering Results

In [51]:
# set number of clusters
kclusters = 5

allvenues_grouped_clustering = allvenues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(allvenues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

In [52]:
# # add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

nyc_toronto_merged = nyc_toronto

# # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nyc_toronto_merged = nyc_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

nyc_toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Toronto,1.0,Fast Food Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Toronto,0.0,Moving Target,Bar,History Museum,Women's Store,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Toronto,1.0,Intersection,Spa,Medical Center,Pizza Place,Mexican Restaurant,Electronics Store,Rental Car Location,Breakfast Spot,Food,Flower Shop
3,Scarborough,Woburn,43.770992,-79.216917,Toronto,0.0,Coffee Shop,Insurance Office,Korean Restaurant,Farm,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
4,Scarborough,Cedarbrae,43.773136,-79.239476,Toronto,0.0,Fried Chicken Joint,Hakka Restaurant,Caribbean Restaurant,Bakery,Bank,Athletics & Sports,Thai Restaurant,Farm,Ethiopian Restaurant,Event Service


In [53]:
nyc_toronto_merged

,Borough,Neighborhood,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Toronto,1.0,Fast Food Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Toronto,0.0,Moving Target,Bar,History Museum,Women's Store,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Toronto,1.0,Intersection,Spa,Medical Center,Pizza Place,Mexican Restaurant,Electronics Store,Rental Car Location,Breakfast Spot,Food,Flower Shop
3,Scarborough,Woburn,43.770992,-79.216917,Toronto,0.0,Coffee Shop,Insurance Office,Korean Restaurant,Farm,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
4,Scarborough,Cedarbrae,43.773136,-79.239476,Toronto,0.0,Fried Chicken Joint,Hakka Restaurant,Caribbean Restaurant,Bakery,Bank,Athletics & Sports,Thai Restaurant,Farm,Ethiopian Restaurant,Event Service
5,Scarborough,Scarborough Village,43.744734,-79.239476,Toronto,2.0,Spa,Playground,Women's Store,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Toronto,0.0,Convenience Store,Department Store,Chinese Restaurant,Coffee Shop,Discount Store,Bus Station,Food & Drink Shop,Food Truck,English Restaurant,Ethiopian Restaurant
7,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Toronto,0.0,Bus Line,Bakery,Intersection,Park,Soccer Field,Metro Station,Bus Station,Fast Food Restaurant,Fish Market,Factory
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,Toronto,0.0,Motel,American Restaurant,Women's Store,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
9,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,Toronto,0.0,General Entertainment,College Stadium,Café,Skating Rink,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service


### Removing null data rows where Foursquare didn't have data

In [71]:
nyc_toronto_merged = nyc_toronto_merged[np.isfinite(nyc_toronto_merged['Cluster Labels'])]

In [73]:
nyc_toronto_merged['Cluster Labels'] = nyc_toronto_merged['Cluster Labels'].astype(np.int64)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [74]:
nyc_toronto_merged

,Borough,Neighborhood,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Toronto,1,Fast Food Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Toronto,0,Moving Target,Bar,History Museum,Women's Store,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Toronto,1,Intersection,Spa,Medical Center,Pizza Place,Mexican Restaurant,Electronics Store,Rental Car Location,Breakfast Spot,Food,Flower Shop
3,Scarborough,Woburn,43.770992,-79.216917,Toronto,0,Coffee Shop,Insurance Office,Korean Restaurant,Farm,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
4,Scarborough,Cedarbrae,43.773136,-79.239476,Toronto,0,Fried Chicken Joint,Hakka Restaurant,Caribbean Restaurant,Bakery,Bank,Athletics & Sports,Thai Restaurant,Farm,Ethiopian Restaurant,Event Service
5,Scarborough,Scarborough Village,43.744734,-79.239476,Toronto,2,Spa,Playground,Women's Store,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Toronto,0,Convenience Store,Department Store,Chinese Restaurant,Coffee Shop,Discount Store,Bus Station,Food & Drink Shop,Food Truck,English Restaurant,Ethiopian Restaurant
7,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Toronto,0,Bus Line,Bakery,Intersection,Park,Soccer Field,Metro Station,Bus Station,Fast Food Restaurant,Fish Market,Factory
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,Toronto,0,Motel,American Restaurant,Women's Store,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
9,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,Toronto,0,General Entertainment,College Stadium,Café,Skating Rink,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service


In [78]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 40.7308619, -73.9871558.


In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_toronto_merged['Latitude'], nyc_toronto_merged['Longitude'], nyc_toronto_merged['Neighborhood'], nyc_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [83]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_TR = location.latitude
longitude_TR = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_TR, longitude_TR))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [84]:
# create map
map_clusters = folium.Map(location=[latitude_TR, longitude_TR], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_toronto_merged['Latitude'], nyc_toronto_merged['Longitude'], nyc_toronto_merged['Neighborhood'], nyc_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Discussion

### Specific Client Use Case

Jimbo Champster is a resident of Toronto and has recently take a job offer in the high-paced city of New York. He has signed with Emmigreation to assist him with his move and make it as pleasant a possible. Part of the new Emmigreation offering is to pinpoint neighbourhoods that will siilar to Jimbo's current neighbourhood to help make the move easier and more enjoyable.

Jimbo's current neighbourhood is that of Parkwoods nighbourhood in Toronto. If we look at the make up of the common amenities in the radius of the area, it is evident that this neighbourhood seems to be suited to someone who likes open spaces, nature and dining (particularly exotic dining such as Spanish and Ethiopian cuisine) See below for the profile of Parkwoods neighbourhood.

In [94]:
nyc_toronto_merged[nyc_toronto_merged['Neighborhood'] == 'Parkwoods']

,Borough,Neighborhood,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,North York,Parkwoods,43.753259,-79.329656,Toronto,3,Food & Drink Shop,Park,Fast Food Restaurant,Women's Store,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service


### Make up of neighbourhood clustered together with Parkwoods but found in New York: Todt Hill

In [96]:
nyc_toronto_merged[nyc_toronto_merged['Neighborhood'] == 'Todt Hill']

,Borough,Neighborhood,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
203,Staten Island,Todt Hill,40.597069,-74.111329,New York,3,Trail,Park,Women's Store,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space


# Conclusion

If Emmigreation were to propose a similar feel neighbourhood to that of Parkwoods for Jimbo, then Todt Hill was found to be in the same cluster. It is evident that Todt Hill has similar features to Parkwoods as it seems to have the same outdoors, open-space kind of profile with exotic dining options available for Jimbo to enjoy. In this case, the k-means clustering algorithm has helped to pinpoint a great location for Jimbo to conisder moving to. This will definitely aid in making the move more pleasant and easy as well as helping Jimbo settle in his new spot much quicker than another neighbourhood which may be drastically different.

# The End